In [4]:
# Importing Libraries for Data Analysis and Visualization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import wordcloud
import nltk
import re
import string
import spacy

sns.set_theme()
%matplotlib inline

import warnings as w
w.filterwarnings('ignore')

In [6]:
airline_df = pd.read_csv('airline_df_cleaned.csv')
airline_df.head(10)

,Airline,Country,Review,Date_Published,Type of Traveller,Seat Type,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Value for Money,Recommended,Origin,Destination,Sentiment
0,Air China,China,los angeles to beijing return. food low qualit...,2012-01-30,Solo Leisure,Economy Class,3,2,2,1,1,3,0,Beijing,Xi'an,Negative
1,Air China,China,round to trip from hong kong to munich. the ma...,2012-01-31,Solo Leisure,Economy Class,1,3,1,1,1,3,0,Beijing,Xi'an,Negative
2,Air China,China,sydney to beijing to paris then rome to beijin...,2012-02-03,Solo Leisure,Economy Class,2,3,1,2,1,3,0,Beijing,Xi'an,Negative
3,Air China,China,london to sydney return via beijing. a cheap f...,2012-02-22,Solo Leisure,Economy Class,4,1,2,4,3,4,0,Beijing,Xi'an,Negative
4,Air China,China,beijing to shanghai. only one check to in desk...,2012-02-28,Solo Leisure,Economy Class,4,4,3,3,3,4,1,Beijing,Xi'an,Positive
5,Air China,China,stockholm to bangkok via beijing. return fligh...,2012-03-13,Solo Leisure,Economy Class,2,1,1,1,1,3,0,Beijing,Xi'an,Negative
6,Air China,China,vancouver to hong kong via beijing. seating wa...,2012-03-19,Solo Leisure,Economy Class,4,3,4,4,3,4,1,Beijing,Xi'an,Positive
7,Air China,China,vancouver to beijing return in economy. my veg...,2012-03-21,Solo Leisure,Economy Class,3,4,4,3,3,4,1,Beijing,Xi'an,Positive
8,Air China,China,hong kong to bjs on a321. old looking plane bu...,2012-03-26,Solo Leisure,Economy Class,4,4,3,2,4,4,1,Beijing,Xi'an,Positive
9,Air China,China,dus to beijing to manila return. 2.5 hours del...,2012-03-26,Solo Leisure,Economy Class,3,1,2,3,2,2,0,Beijing,Xi'an,Negative


In [7]:
airline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15203 entries, 0 to 15202
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Airline                 15203 non-null  object
 1   Country                 15203 non-null  object
 2   Review                  15203 non-null  object
 3   Date_Published          15203 non-null  object
 4   Type of Traveller       15203 non-null  object
 5   Seat Type               15203 non-null  object
 6   Seat Comfort            15203 non-null  int64 
 7   Cabin Staff Service     15203 non-null  int64 
 8   Food & Beverages        15203 non-null  int64 
 9   Inflight Entertainment  15203 non-null  int64 
 10  Ground Service          15203 non-null  int64 
 11  Value for Money         15203 non-null  int64 
 12  Recommended             15203 non-null  int64 
 13  Origin                  14956 non-null  object
 14  Destination             14994 non-null  object
 15  Se

In [8]:
airline_df.describe(include='all')

,Airline,Country,Review,Date_Published,Type of Traveller,Seat Type,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Value for Money,Recommended,Origin,Destination,Sentiment
count,15203,15203,15203,15203,15203,15203,15203.000000,15203.000000,15203.000000,15203.000000,15203.000000,15203.000000,15203.00000,14956,14994,15203
unique,19,9,15202,3718,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,730,711,2
top,Emirates,India,trip verified very attentive staff at airport ...,2015-01-14,Solo Leisure,Economy Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Delhi,Mumbai,Positive
freq,2263,3636,2,39,6218,11368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1175,1021,8905
mean,NaN,NaN,NaN,NaN,NaN,NaN,3.370979,3.502204,3.148655,3.227587,3.074854,3.365586,0.58574,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,1.376109,1.547916,1.475227,1.389971,1.549448,1.527154,0.49261,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,0.00000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,4.000000,3.000000,3.000000,3.000000,4.000000,1.00000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,5.000000,4.000000,4.000000,4.000000,5.000000,1.00000,NaN,NaN,NaN


In [10]:
for i in airline_df.columns:
    print(i, airline_df[i].unique())

Airline ['Air China' 'Air India' 'Air India Express' 'Airasia India'
 'Ana All Nippon Airways' 'China Eastern Airlines'
 'China Southern Airlines' 'Emirates' 'Goair' 'Indigo Airlines'
 'Japan Airlines' 'Jet Airways' 'Korean Air' 'Lion Air' 'Qatar Airways'
 'Singapore Airlines' 'Spicejet' 'Vietjetair' 'Vistara']
Country ['China' 'India' 'Japan' 'UAE' 'South Korea' 'Indonesia' 'Qatar'
 'Singapore' 'Vietnam']
Review ['los angeles to beijing return. food low quality staff appeared when time for mandatory service large screen viewing on main bulkhead without sound on both trips. they were older 747s. it seems staff had something to say at irregular intervals making it hard to try and sleep but they were very good at making business transactions ignoring everyone so they could count money and look at receipts. you want a cheap very basic airline trip to china then this is the airline for you.'
 'round to trip from hong kong to munich. the main reason for flying air china was no surprises the

## Questions for Data Analysis on Airline Reviews, Ratings and Sentiment (Dashboards)
> #### 1. What is the distribution of Seat_Comfort, Cabin_Service, Food_Bev, Entertainment, Ground_Service, and Value_for_Money of each Airline?
> #### 2. What is the distribution of Type of Traveller, Seat Type, Route, and Date Flown of each airline?
> #### 3. What is the distribution of Overall Rating of each airline?
> #### 4. What is the distribution of Sentiment of each airline?
> #### 5. What is the distribution of Overall Rating of each airline by Type of Traveller, Seat Type, Country, Airline ?

In [ ]:
fig = px.histogram(airline_df, x="Sentiment", color="Sentiment", title="Sentiment Distribution",labels={'Sentiment':'Sentiment','count':'Count'},barmode='group',width=800)
fig.show()

In [ ]:
# Sentiment of each airline
fig = px.histogram(airline_df, x="Airline", color="Sentiment",
                title="Airline Wise Sentiment Distribution",
                barmode='group', histfunc='count', height=650)
fig_widget = go.FigureWidget(fig)
fig_widget
fig.show()

In [ ]:
# Pie chart of sentiment distribution
for col in ["Seat Comfort",
        "Cabin Staff Service",
        "Food & Beverages",
        "Inflight Entertainment",
        "Ground Service",
        "Value for Money",
        "Sentiment",
    ]:
        plt.pie(airline_df[col].value_counts(), labels=airline_df[col].value_counts().index,autopct='%1.1f%%', shadow=True, startangle=90)
        plt.title(col)
        plt.axis('equal')
        plt.legend()
        plt.show()

In [ ]:
# Overall Rating of each airline
fig = px.histogram(airline_df, x="Airline", color="Overall_Rating", title="Airline Wise Overall Rating Distribution",barmode='group', histfunc='count', height=650) 
fig_widget = go.FigureWidget(fig)
fig_widget
fig.show()

In [ ]:
airline_df